In [ ]:
import pandas as pd

# Load the uploaded CSV data to check its structure
file_path = 'corrected_adj_close_data.csv'
data = pd.read_csv(file_path)
data.head()


In [ ]:
# Convert the 'Date' column to datetime format and set it as the index
data['Date'] = pd.to_datetime(data['Date'])
data.set_index('Date', inplace=True)

# Calculate daily returns
daily_returns = data.pct_change().dropna(how='all')  # drop days where all returns are NaN

# Show the first few rows of daily returns
daily_returns.head()


In [ ]:
# Compute the covariance matrix of the daily returns
covariance_matrix = daily_returns.cov()

# Show the covariance matrix
covariance_matrix


In [ ]:
# Define the periods for analysis
periods = {
    "Financial Crisis": ("2007-12-01", "2009-06-30"),
    "Bull Market 2009-2020": ("2009-06-01", "2020-02-29"),
    "Bull Market 2020-2023": ("2020-04-01", "2022-12-31")
}

# Initialize a dictionary to store results
period_data = {}

# Segment data, calculate mean returns and covariance matrices for each period
for period_name, (start_date, end_date) in periods.items():
    period_data[period_name] = {}
    # Filter data for the period
    period_data[period_name]['returns'] = daily_returns.loc[start_date:end_date]
    # Calculate mean returns
    period_data[period_name]['mean_returns'] = period_data[period_name]['returns'].mean()
    # Calculate covariance matrix
    period_data[period_name]['covariance'] = period_data[period_name]['returns'].cov()

# Output the calculated mean returns for the Financial Crisis period to check
period_data['Financial Crisis']['mean_returns'], period_data['Financial Crisis']['covariance']


In [ ]:
# Output the mean returns and covariance matrices for the remaining periods
bull_market_2009_2020_mean_returns = period_data['Bull Market 2009-2020']['mean_returns']
bull_market_2009_2020_covariance = period_data['Bull Market 2009-2020']['covariance']

bull_market_2020_2023_mean_returns = period_data['Bull Market 2020-2023']['mean_returns']
bull_market_2020_2023_covariance = period_data['Bull Market 2020-2023']['covariance']

(bull_market_2009_2020_mean_returns, bull_market_2009_2020_covariance,
 bull_market_2020_2023_mean_returns, bull_market_2020_2023_covariance)


In [ ]:
import numpy as np

# Function to calculate Risk Parity weights
def risk_parity_weights(covariance_matrix):
    # Calculate inverse variance weights
    iv = 1 / np.diag(covariance_matrix)
    iv_weights = iv / sum(iv)
    return iv_weights

# Function to calculate Maximum Diversification Portfolio weights
def mdp_weights(covariance_matrix):
    inv_covar = np.linalg.inv(covariance_matrix)
    ones = np.ones(len(covariance_matrix))
    mdp_w = inv_covar @ ones / (ones.T @ inv_covar @ ones)
    return mdp_w

# Function to calculate Mean-Variance Optimization weights
def mvo_weights(mean_returns, covariance_matrix):
    inv_covar = np.linalg.inv(covariance_matrix)
    mvo_w = inv_covar @ mean_returns / (mean_returns.T @ inv_covar @ mean_returns)
    return mvo_w

# Function to calculate Hierarchical Risk Parity weights (simplified)
from scipy.cluster.hierarchy import linkage, leaves_list

def hrp_weights(covariance_matrix):
    # Perform hierarchical clustering
    corr = covariance_matrix.corr()
    row_linkage = linkage(corr, method='single')
    sorted_indices = leaves_list(row_linkage)
    
    # Allocate weights based on inverse variance clustering
    sorted_cov = covariance_matrix.iloc[sorted_indices, sorted_indices]
    iv = 1 / np.diag(sorted_cov)
    hrp_w = iv / sum(iv)
    return hrp_w[sorted_indices.argsort()]  # Reorder to original asset order

# Calculate portfolio weights for each method and each period
portfolio_weights = {}
for period_name, data in period_data.items():
    cov_matrix = data['covariance'].dropna(axis=0, how='all').dropna(axis=1, how='all')
    mean_returns = data['mean_returns'][cov_matrix.columns]
    
    portfolio_weights[period_name] = {
        'RP': risk_parity_weights(cov_matrix),
        'MDP': mdp_weights(cov_matrix),
        'MVO': mvo_weights(mean_returns, cov_matrix),
        'HRP': hrp_weights(cov_matrix)
    }

portfolio_weights['Financial Crisis']['RP'], portfolio_weights['Financial Crisis']['MDP']  # Show RP and MDP weights for Financial Crisis as an example


In [ ]:
def portfolio_performance(weights, returns):
    """ Calculate portfolio performance metrics: Sharpe Ratio, Maximum Drawdown, and Volatility. """
    # Calculate portfolio returns
    portfolio_returns = returns.dot(weights)
    
    # Calculate Sharpe Ratio (assuming risk-free rate is 0 for simplicity)
    sharpe_ratio = portfolio_returns.mean() / portfolio_returns.std() * np.sqrt(252)  # annualized
    
    # Calculate Maximum Drawdown
    cumulative_returns = (1 + portfolio_returns).cumprod()
    peak = cumulative_returns.cummax()
    drawdown = (cumulative_returns - peak) / peak
    max_drawdown = drawdown.min()
    
    # Calculate Volatility (annualized)
    volatility = portfolio_returns.std() * np.sqrt(252)
    
    return sharpe_ratio, max_drawdown, volatility

# Re-calculate and store the metrics for each portfolio in each period
portfolio_metrics = {}
for period_name, data in period_data.items():
    cov_matrix = data['covariance'].dropna(axis=0, how='all').dropna(axis=1, how='all')
    valid_returns = data['returns'][cov_matrix.columns]  # Filter returns for valid columns
    portfolio_metrics[period_name] = {}
    for portfolio_name, weights in portfolio_weights[period_name].items():
        weights_aligned = weights / weights.sum()  # Normalize weights to sum to 1
        sharpe, mdd, vol = portfolio_performance(weights_aligned, valid_returns)
        portfolio_metrics[period_name][portfolio_name] = {
            'Sharpe Ratio': sharpe,
            'Maximum Drawdown': mdd,
            'Volatility': vol
        }

# Show the metrics for the Financial Crisis period as an example
portfolio_metrics['Financial Crisis']


In [ ]:
# Show the metrics for the Bull Market 2009-2020 and Bull Market 2020-2023 periods
portfolio_metrics['Bull Market 2009-2020'], portfolio_metrics['Bull Market 2020-2023']


In [ ]:
from scipy.stats import ttest_rel

# Prepare data for t-tests
data_for_tests = {
    'Sharpe Ratio': {},
    'Volatility': {}
}

for period, strategies in portfolio_metrics.items():
    for metric in ['Sharpe Ratio', 'Volatility']:
        data_for_tests[metric][period] = {
            'HRP': strategies['HRP'][metric],
            'RP': strategies['RP'][metric],
            'MDP': strategies['MDP'][metric],
            'MVO': strategies['MVO'][metric]
        }

# Perform paired t-tests for Sharpe Ratio and Volatility
test_results = {
    'Sharpe Ratio': {},
    'Volatility': {}
}

for metric in ['Sharpe Ratio', 'Volatility']:
    test_results[metric] = {}
    for period in data_for_tests[metric]:
        hrp = data_for_tests[metric][period]['HRP']
        rp = data_for_tests[metric][period]['RP']
        mdp = data_for_tests[metric][period]['MDP']
        mvo = data_for_tests[metric][period]['MVO']
        
        # Paired t-tests comparing HRP against each of RP, MDP, and MVO
        test_results[metric][period] = {
            'HRP vs RP': ttest_rel([hrp], [rp])[1],
            'HRP vs MDP': ttest_rel([hrp], [mdp])[1],
            'HRP vs MVO': ttest_rel([hrp], [mvo])[1]
        }

test_results
